In [3]:
# default
from pprint import pprint
import natsort
import os
import pickle as pkl

# data structure
import numpy as np
from PIL import Image

# Deep Learning
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

from torchvision import transforms
from torchvision import models
from torchvision.models import resnet50
from torchvision.models import efficientnet_v2_s, efficientnet_v2_m, efficientnet_v2_l

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import module.skin_disease_model as sdm
from module.data_processing import mkadir

TRAIN = 0
VALID = 1
data_type = ["train", "valid"]
dt = data_type[TRAIN]

project_path = "D:/Capstone/"
original_data_path = os.path.join(project_path, "dataset/", "original/", dt)
RoI_data_path = os.path.join(project_path, "dataset/", "RoI/", dt)
box_data_path = os.path.join(project_path, "dataset/", "box/256x256/", dt)
crop_data_path = os.path.join(project_path, "dataset/", "crop/", dt)
RZ_data_path = os.path.join(project_path, "dataset/", "ratio_zero-padding/", dt)
RZ_LANCZOS4_data_path = os.path.join(project_path, "dataset/", "RZ_LANCZOS4/", dt)

data_path_namelist = ["original", "RoI", "box", "crop", "ratio_zero-padding", "RZ_LANCZOS4"]
data_path_list = [original_data_path, RoI_data_path, box_data_path, crop_data_path, RZ_data_path, RZ_LANCZOS4_data_path]

print("DATA TYPE:", dt, end="\n\n")
for i, data_path in enumerate(data_path_list):
    print(data_path_namelist[i] + " data folder")
    try:
        folder = natsort.natsorted(os.listdir(data_path))
        print(folder, end="\n\n")
    except:
        print("there is no directory.") 

DATA TYPE: train

original data folder
['A1', 'A2', 'A3', 'A4', 'A5', 'A6']

RoI data folder
['A1', 'A2', 'A3', 'A4', 'A5', 'A6']

box data folder
there is no directory.
crop data folder
there is no directory.
ratio_zero-padding data folder
there is no directory.
RZ_LANCZOS4 data folder
['A1', 'A2', 'A3', 'A4', 'A5', 'A6']



load dataset

In [4]:
size = 224
dataset_path = "/".join(RZ_LANCZOS4_data_path.split("/")[:-1]) + "/"

# 이미지 불러오기
with open(os.path.join(dataset_path, "224x224_train_data.pkl"), "rb") as pkl_file:
  train_x, train_y = pkl.load(pkl_file)

with open(os.path.join(dataset_path, "224x224_valid_data.pkl"), "rb") as pkl_file:
  valid_x, valid_y = pkl.load(pkl_file)

print("train_x length: ", len(train_x))
print("train_x type: ", type(train_x))
print("train_y length: ", train_y.shape)
print("train_y type: ", type(train_y))
print("valid_x length: ", len(valid_x))
print("valid_x type: ", type(valid_x))
print("valid_y length: ", valid_y.shape)
print("valid_y type: ", type(valid_y))

train_x length:  52914
train_x type:  <class 'list'>
train_y length:  (52914,)
train_y type:  <class 'numpy.ndarray'>
valid_x length:  6645
valid_x type:  <class 'list'>
valid_y length:  (6645,)
valid_y type:  <class 'numpy.ndarray'>


torch

In [5]:
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [6]:
train_dataset = sdm.Skin_Disease_Dataset(train_x, train_y, train_transforms)
valid_dataset = sdm.Skin_Disease_Dataset(valid_x, valid_y, test_transforms)
train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [7]:
model_path = os.path.join(project_path, "model/", "2023-03-16/")
mkadir(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = sdm.Skin_Distinction_Model(model=efficientnet_v2_s(weights="DEFAULT"),
                                   out_features=6,
                                   device=device,
                                   save_path=model_path).to(device)

pprint(summary(model, input_size=(1, 3, 224, 224), verbose=0))

Layer (type:depth-idx)                                       Output Shape              Param #
Skin_Distinction_Model                                       [1, 6]                    --
├─EfficientNet: 1-1                                          [1, 1000]                 --
│    └─Sequential: 2-1                                       [1, 1280, 7, 7]           --
│    │    └─Conv2dNormActivation: 3-1                        [1, 24, 112, 112]         696
│    │    └─Sequential: 3-2                                  [1, 24, 112, 112]         10,464
│    │    └─Sequential: 3-3                                  [1, 48, 56, 56]           303,552
│    │    └─Sequential: 3-4                                  [1, 64, 28, 28]           589,184
│    │    └─Sequential: 3-5                                  [1, 128, 14, 14]          917,680
│    │    └─Sequential: 3-6                                  [1, 160, 14, 14]          3,463,840
│    │    └─Sequential: 3-7                                  [1, 256

In [8]:
# 0.000213388
# optimizer = optim.SGD(model.parameters(), lr=7.5e-7, momentum=0.999, weight_decay=1e-5, nesterov=True)

optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss().to(device)
epochs = 100

In [9]:
print(device)
parameters = {
    'Train_DataLoader' : train_data_loader,
    'Valid_DataLoader' : valid_data_loader,
    'Optimizer'        : optimizer,
    'Loss_function'    : criterion,
    'Epochs'           : epochs
}

cuda


In [10]:
history = model.fit(parameters)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1/100
Train - Loss: 1.232733, Accuracy: 52.30
Valid - Loss: 1.315885, Accuracy: 48.52
Save model, validation loss: 1.31588453742174
Save model, validation acc: 48.51914005805516
0 epoch end lr:  0.0001
Epoch: 2/100
Train - Loss: 0.998617, Accuracy: 61.88
Valid - Loss: 1.253559, Accuracy: 52.24
Save model, validation loss: 1.2535587537747164
Save model, validation acc: 52.235747006531206
1 epoch end lr:  0.0001
Epoch: 3/100
Train - Loss: 0.866309, Accuracy: 67.07
Valid - Loss: 1.237729, Accuracy: 54.89
Save model, validation loss: 1.237729155100309
Save model, validation acc: 54.89188361756168
2 epoch end lr:  0.0001
Epoch: 4/100
Train - Loss: 0.743351, Accuracy: 72.09
Valid - Loss: 1.274365, Accuracy: 56.32
Save model, validation acc: 56.31973421625544
3 epoch end lr:  0.0001
Epoch: 5/100
Train - Loss: 0.633359, Accuracy: 76.34
Valid - Loss: 1.334363, Accuracy: 57.12
Save model, validation acc: 57.118559506531206
4 epoch end lr:  0.0001
Epoch: 6/100
Train - Loss: 0.529021, Accur